In [1]:
#Loading packages
import seaborn as sns
import pandas as pd
import numpy as np
from matplotlib import pyplot
import matplotlib.pyplot as plt
import string
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
import sqlite3
from sqlalchemy import create_engine
from sklearn.preprocessing import MinMaxScaler
from geopy.distance import geodesic
import joblib

import warnings
warnings.filterwarnings('ignore')

## Part 7 - Executing with live data

We will start by importing our live data from a csv

In [2]:
#Importing data
live_data = pd.read_csv('split100.csv')
live_data.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,30284277,"City Flat; Sleeps 6; Steps to Zoo, Near Downtown",142047837,Kate,NaN,Lincoln Park,41.92070,-87.64236,Entire home/apt,154,3,48,2020-03-27,2.40,2,0
1,20235190,302 Sweet Sensation room Andersonville Free wifi,127235673,Alex,NaN,Edgewater,41.98532,-87.67212,Private room,36,1,37,2020-07-23,0.97,11,351
2,18123049,Fun & Funky Musician Condo (Entire Home),36370758,Rebecca,NaN,Irving Park,41.96015,-87.71004,Entire home/apt,120,3,16,2019-08-12,0.39,1,0
3,27398743,"Modern Urban Suite - West Town, Centrally Located",29622436,Dina,NaN,West Town,41.89550,-87.66042,Entire home/apt,79,2,81,2020-08-29,3.11,1,0
4,25184769,Sonder | 943 Crosby | Lively 1BR,12243051,Sonder,NaN,Near North Side,41.89917,-87.64299,Entire home/apt,84,30,4,2020-09-05,0.20,47,79


Now, we will import our clean_data_table from postgresql

In [3]:
#Information from PostgreSQL
host = r'localhost' 
db = r'MSDS610' 
user = r'postgres' 
pw = r'82328' 
port = r'5432' 
schema = r'cleaned' 

In [4]:
#Creating a connection
db_conn = create_engine("postgresql://{}:{}@{}:{}/{}".format(user, pw, host, port, db))

In [5]:
#Listing the tables
sql="select tables.table_name from information_schema.tables where (table_schema ='"+schema+"')order by 1;"
tbl_df = pd.read_sql(sql, db_conn, index_col=None)
tbl_df

,table_name
0,clean_data_table


In [6]:
#Table name
table_name= r'clean_data_table'

In [7]:
#Verifying SQL database
sql=f'SELECT * FROM "{schema}"."{table_name}"'
clean_data_table= pd.read_sql(sql, db_conn, index_col=None)

In [8]:
clean_data_table

,Field_name,Manipulation_type,Numeric_values
0,id,Drop,None
1,name,Drop,None
2,host_id,Drop,None
3,neighbourhood_group,Drop,None
4,last_review,Drop,None
5,reviews_per_month,NaN_replace,0
6,price,Outliers,"(0,465)"
7,neighbourhood,Drop,None
8,room_type,encoded,None
9,price_per_year,new_feature,None


We will proceed to clean the live_data with the guidence of the clean_data_table.

We will start by dropping some columns.

In [9]:
#Dropping neighbourhood_group
live_data = live_data.drop(columns=['neighbourhood_group', 'last_review', 'host_name', 'name', 'host_id', 'id'])
live_data.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Lincoln Park,41.92070,-87.64236,Entire home/apt,154,3,48,2.40,2,0
1,Edgewater,41.98532,-87.67212,Private room,36,1,37,0.97,11,351
2,Irving Park,41.96015,-87.71004,Entire home/apt,120,3,16,0.39,1,0
3,West Town,41.89550,-87.66042,Entire home/apt,79,2,81,3.11,1,0
4,Near North Side,41.89917,-87.64299,Entire home/apt,84,30,4,0.20,47,79


Now, we will fill our missing values with 0.

In [10]:
#Replacing nan with 0 for reviews_per_month column
live_data['reviews_per_month'] = live_data['reviews_per_month'].fillna(0)

We will trim the price column for numbers above 0, but below 465.

In [11]:
live_data = live_data[(live_data['price']> 0) & (live_data['price']< 465)]
live_data

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365
0,Lincoln Park,41.92070,-87.64236,Entire home/apt,154,3,48,2.40,2,0
1,Edgewater,41.98532,-87.67212,Private room,36,1,37,0.97,11,351
2,Irving Park,41.96015,-87.71004,Entire home/apt,120,3,16,0.39,1,0
3,West Town,41.89550,-87.66042,Entire home/apt,79,2,81,3.11,1,0
4,Near North Side,41.89917,-87.64299,Entire home/apt,84,30,4,0.20,47,79
...,...,...,...,...,...,...,...,...,...,...
95,New City,41.79449,-87.66167,Private room,39,3,2,0.18,7,324
96,Loop,41.86920,-87.63190,Entire home/apt,196,2,36,2.36,45,344
97,West Town,41.89099,-87.68188,Entire home/apt,250,1,0,0.00,4,324
98,West Town,41.89823,-87.68656,Private room,71,2,51,1.80,1,89


We now start by creating some engineering features.

In [12]:
#Creating a feature price per night
live_data['price_per_year'] = live_data['price']*live_data['availability_365']
live_data.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_per_year
0,Lincoln Park,41.92070,-87.64236,Entire home/apt,154,3,48,2.40,2,0,0
1,Edgewater,41.98532,-87.67212,Private room,36,1,37,0.97,11,351,12636
2,Irving Park,41.96015,-87.71004,Entire home/apt,120,3,16,0.39,1,0,0
3,West Town,41.89550,-87.66042,Entire home/apt,79,2,81,3.11,1,0,0
4,Near North Side,41.89917,-87.64299,Entire home/apt,84,30,4,0.20,47,79,6636


In [13]:
#Creating a 'distance_to_city_center' feature (Using Chicago city center latitude, longitude)
city_center = (41.8781, -87.6298) 
live_data['distance_to_city_center'] = live_data.apply(lambda row: geodesic((row['latitude'], row['longitude']), city_center).km, axis=1)

live_data.head()

,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_per_year,distance_to_city_center
0,Lincoln Park,41.92070,-87.64236,Entire home/apt,154,3,48,2.40,2,0,0,4.845067
1,Edgewater,41.98532,-87.67212,Private room,36,1,37,0.97,11,351,12636,12.415616
2,Irving Park,41.96015,-87.71004,Entire home/apt,120,3,16,0.39,1,0,0,11.285465
3,West Town,41.89550,-87.66042,Entire home/apt,79,2,81,3.11,1,0,0,3.192758
4,Near North Side,41.89917,-87.64299,Entire home/apt,84,30,4,0.20,47,79,6636,2.583648


We will proceed with dropping the neighbourhood column as part of our data modification process. However, we encountered an issue with this column. Unfortunately, the live data does not contain all the neighbourhoods that were not part of the original data. This inconsistency needed to be addressed before continuing with the analysis, therefore we decided to remove neighbourhood from our original data to train the model as well as live_data.

In [14]:
#Dropping neighbourhood_group
live_data = live_data.drop(columns=['neighbourhood'])
live_data.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_per_year,distance_to_city_center
0,41.92070,-87.64236,Entire home/apt,154,3,48,2.40,2,0,0,4.845067
1,41.98532,-87.67212,Private room,36,1,37,0.97,11,351,12636,12.415616
2,41.96015,-87.71004,Entire home/apt,120,3,16,0.39,1,0,0,11.285465
3,41.89550,-87.66042,Entire home/apt,79,2,81,3.11,1,0,0,3.192758
4,41.89917,-87.64299,Entire home/apt,84,30,4,0.20,47,79,6636,2.583648


To finalize our cleaning, we will proceed to encode room_type column.

In [15]:
#Encoding neighbourhood and room_type data
live_data_encoded = pd.get_dummies(live_data, columns=['room_type'])
live_data_encoded.head()

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_per_year,distance_to_city_center,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,41.92070,-87.64236,154,3,48,2.40,2,0,0,4.845067,True,False,False,False
1,41.98532,-87.67212,36,1,37,0.97,11,351,12636,12.415616,False,False,True,False
2,41.96015,-87.71004,120,3,16,0.39,1,0,0,11.285465,True,False,False,False
3,41.89550,-87.66042,79,2,81,3.11,1,0,0,3.192758,True,False,False,False
4,41.89917,-87.64299,84,30,4,0.20,47,79,6636,2.583648,True,False,False,False


We want to ensure that the data was cleaned.

In [16]:
live_data_encoded

,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_per_year,distance_to_city_center,room_type_Entire home/apt,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,41.92070,-87.64236,154,3,48,2.40,2,0,0,4.845067,True,False,False,False
1,41.98532,-87.67212,36,1,37,0.97,11,351,12636,12.415616,False,False,True,False
2,41.96015,-87.71004,120,3,16,0.39,1,0,0,11.285465,True,False,False,False
3,41.89550,-87.66042,79,2,81,3.11,1,0,0,3.192758,True,False,False,False
4,41.89917,-87.64299,84,30,4,0.20,47,79,6636,2.583648,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,41.79449,-87.66167,39,3,2,0.18,7,324,12636,9.656507,False,False,True,False
96,41.86920,-87.63190,196,2,36,2.36,45,344,67424,1.003785,True,False,False,False
97,41.89099,-87.68188,250,1,0,0.00,4,324,81000,4.553587,True,False,False,False
98,41.89823,-87.68656,71,2,51,1.80,1,89,6319,5.214502,False,False,True,False


We will load the saved model created with the original dataset.

In [17]:
# Load the saved model
model_filename = "iris_rf_model.joblib"
loaded_model = joblib.load(model_filename)

In [18]:
#Create predictions for live data
predictions = loaded_model.predict(live_data_encoded)

## Part 8 - Storing the Predictions

Now, we will proceed to create prediction per each row in the live data

In [19]:
#Create a predictions column in the live data
live_data['predictions'] = predictions
live_data

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,price_per_year,distance_to_city_center,predictions
0,41.92070,-87.64236,Entire home/apt,154,3,48,2.40,2,0,0,4.845067,1.035054
1,41.98532,-87.67212,Private room,36,1,37,0.97,11,351,12636,12.415616,0.355128
2,41.96015,-87.71004,Entire home/apt,120,3,16,0.39,1,0,0,11.285465,1.292448
3,41.89550,-87.66042,Entire home/apt,79,2,81,3.11,1,0,0,3.192758,0.632429
4,41.89917,-87.64299,Entire home/apt,84,30,4,0.20,47,79,6636,2.583648,0.553062
...,...,...,...,...,...,...,...,...,...,...,...,...
95,41.79449,-87.66167,Private room,39,3,2,0.18,7,324,12636,9.656507,0.547052
96,41.86920,-87.63190,Entire home/apt,196,2,36,2.36,45,344,67424,1.003785,1.192105
97,41.89099,-87.68188,Entire home/apt,250,1,0,0.00,4,324,81000,4.553587,2.001949
98,41.89823,-87.68656,Private room,71,2,51,1.80,1,89,6319,5.214502,0.570517


We will load our data into the "analytics" schema, live_data table.

In [20]:
table_name = r'live_data'
schema = r'analytics' 

live_data.to_sql(table_name, con=db_conn, if_exists='replace', index=False, schema=schema, chunksize=1000, method='multi')

98

Predictions saved in a table in the "analytics" schema.

<img align="left" style="padding-right:15px;" src="live_data_predictions-screenshot.png" width=350><br>

## Part 9 - Insights

Based on the analysis of the data, we can determine an optimal pricing strategy for renting out a property in Chicago. The model’s strong performance, with an R-squared value of 0.9757 and a low Mean Squared Error of 0.0097, indicates that it accurately predicts pricing based on key features. The most important factors influencing the nightly rate include price, price_per_year, availability_365, room_type, and distance_to_city_center.

* Price is the most significant feature, closely tied to other variables like the price_per_year and availability_365, which reflect the overall potential earnings and booking frequency of a listing.
* Room_type also plays a crucial role, as different types of spaces can affect the nightly rate.
* The distance_to_city_center helps set the price by reflecting how far the property is from key attractions in the city, with closer properties likely commanding a higher price.
    
The pricing strategy should be based on the type of property wanting to be purchased, its location relative to the city center, its availability throughout the year, and whether it's a room or an entire unit. With this data-driven approach, we can optimize our nightly rate and maximize the return on investment.

## Part 10 - Reflection

Splitting the dataset initially and then building the model using the remaining data posed some challenges. When trying to train the model with live data, I encountered issues with the "neighbourhood" column. Some neighborhoods in the live data weren’t present in the training dataset, while new neighborhoods from the live data didn’t exist in the model's original data. As a result, I had to remove the "neighbourhood" column to resolve the discrepancies. 

Unfortunately, I wasn’t able to follow the suggested approach using UDFs (User-defined Functions), so I opted to apply the same data cleaning process as I did for the training dataset. 

I do have a better understanding of the steps to deploy a model and how important high-quality data, pre-processing, and feature engineering are crucial factors that could have an impact in the model performance.
